In [7]:
from candidate import Phrase

phrases = "New York, windows, catwalk, teardrops, commuters, curved, floor, shape, video, walls, Grand Central Terminal"
phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
context = "Inside, it’s even wackier: curved walls, windows in the shape of teardrops, and a catwalk with a tiny video screen embedded in the floor that shows an endless loop of antlike commuters rushing through Grand Central Terminal in New York."

bl = BaselineLinker()

# for phrase, candidate in bl.link(context, phrases):
#     print(phrase.text, candidate.link)

bl.link_ttl?

In [ ]:
candidate

In [ ]:
sorted(cd)
for c in cd:
    print(".", hash(c))
    

In [ ]:
%load_ext autoreload
%autoreload 2

# DQL

In [ ]:
query_and_save('allUris:"barackobama.com"', "data/all-uris.json")
query_and_save('wikipediaUri:"en.wikipedia.org/wiki/Barack_Obama"', "data/wiki-uri.json")
query_and_save('allUris:"en.wikipedia.org/wiki/Barack\_Obama"', "data/all-uris-wiki.json")
query_and_save('origins:"en.wikipedia.org/wiki/Barack_Obama"', "data/origins.json")

In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [ ]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")